In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
sent = ('휴일 인 오늘 도 서쪽 을 중심 으로 폭염 이 이어졌는데요, 내일 은 반가운 비 소식 이 있습니다.','폭염 을 피해서 휴일 에 놀러왔다가 갑작스런 비 로 인해 망연자실 하고 있습니다.')

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sent)

idf = tfidf_vectorizer.idf_
print(dict(zip(tfidf_vectorizer.get_feature_names(), idf)))

{'갑작스런': 1.4054651081081644, '내일': 1.4054651081081644, '놀러왔다가': 1.4054651081081644, '망연자실': 1.4054651081081644, '반가운': 1.4054651081081644, '서쪽': 1.4054651081081644, '소식': 1.4054651081081644, '오늘': 1.4054651081081644, '으로': 1.4054651081081644, '이어졌는데요': 1.4054651081081644, '인해': 1.4054651081081644, '있습니다': 1.0, '중심': 1.4054651081081644, '폭염': 1.0, '피해서': 1.4054651081081644, '하고': 1.4054651081081644, '휴일': 1.0}


### 자카드 유사도
> 두 문장을 각각 단어의 집합으로 만든 뒤 두 집합을 통해 유사도를 측정하는 방식

### 코사인 유사도
> 두 개의 벡터값에서 코사인 각도를 구하는 방법  
단순히 좌표 상의 거리를 구하는 다른 유사도 측정 방법에 비해 코사인 유사도는 말 그대로 벡터간 각도를 구하는 것이기 때문에 방향성의 개념이 더해진다.

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

array([[0.17952266]])

###  유클리디안 유사도(L2 거리)
> n차원 공간에서 두 점 사이의 최단 거리를 구하는 접근법  

- 거리를 뜻하기 때문에 제한이 없다. 따라서 벡터를 정규화해서 유사도를 측정하면 0~1사이의 값을 갖게 된다.
- L1 유사도를 사용하자.
    - 각 벡터 안의 요소 값들을 모두 더한 것이 크기가 1이 되도록 벡터의 크기를 조절하는 방법

In [9]:
from sklearn.metrics.pairwise import euclidean_distances

euclidean_distances(tfidf_matrix[0:1], tfidf_matrix[1:2])

array([[1.28099753]])

In [5]:
import numpy as np

def l1_normalize(v):
    norm = np.sum(v)
    return v / norm

tfidf_norm_l1 = l1_normalize(tfidf_matrix)
euclidean_distances(tfidf_norm_l1[0:1], tfidf_norm_l1[1:2])

array([[0.20491229]])

### 맨하탄 유사도(L1 거리)
> 사각형 격자로 이루어진 지도에서 출발점부터 도착점까지 가로지르지 않고 갈 수 있는 최단거리

- L1 정규화 방법을 사용한 뒤 유사도를 측정해보자.

In [6]:
from sklearn.metrics.pairwise import manhattan_distances

manhattan_distances(tfidf_norm_l1[0:1], tfidf_norm_l1[1:2])

array([[0.77865927]])

측정 방법에 따라 유사도가 크게 달라질 수 있으므로 의도하고자 하는 방향에 맞게 측정 방법을 고르도록 하자.

### 데이터 이해하기

In [7]:
import os
import re

import pandas as pd
import tensorflow as tf
from tensorflow.keras import utils

data_set = tf.keras.utils.get_file(fname='imdb.tar.gz',origin='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz',
                                  extract=True)

84131840/84125825 [==============================] - 26s 0us/step


KeyboardInterrupt: 

In [ ]:
def directory_data(directory):
    data = {}
    data["review"] = []
    for file_path in os.listdir(directory):
        with open(os.path.join(directory, file_path), "r", encoding='utf-8') as file:
            data["review"].append(file.read())
            
    return pd.DataFrame.from_dict(data)

In [ ]:
def data(directory):
    pos_df = directory_data(os.path.join(directory, "pos"))
    neg_df = directory_data(os.path.join(directory, "neg"))
    pos_df["sentiment"] = 1
    neg_df["sentiment"] = 0
    
    return pd.concat([pos_df, neg_df])

In [ ]:
train_df = data(os.path.join(os.path.dirname(data_set), "aclImdb", "train"))
test_df = data(os.path.join(os.path.dirname(data_set), "aclImdb", "test"))

In [ ]:
train_df.head()

In [ ]:
reviews = list(train_df['review'])

In [ ]:
tokenized_reviews = [r.split() for r in reviews]
review_len_by_token = [len(t) for t in tokenized_reviews]
review_len_by_eumjeol = [len(s.replace(' ','')) for s in reviews]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,5))
plt.hist(review_len_by_token, bins = 50, alpha = 0.5, color = 'r', label = 'word')
plt.hist(review_len_by_eumjeol, bins = 50, alpha = 0.5, color = 'b', label = 'alphabet')
plt.yscale('log', nonposy='clip')

plt.title('Review Length Histogram')
plt.xlabel('Review Length')
plt.ylabel('Number of Reviews')

In [ ]:
print('문장 최대길이: {}'.format(np.max(review_len_by_token)))
print('문장 최소길이: {}'.format(np.min(review_len_by_token)))
print('문장 평균길이: {:.2f}'.format(np.mean(review_len_by_token)))
print('문장 길이 표준편차: {:.2f}'.format(np.std(review_len_by_token)))
print('문장 중간길이: {}'.format(np.median(review_len_by_token)))

print('제 1 사분위 길이: {}'.format(np.percentile(review_len_by_token, 25)))
print('제 3 사분위 길이: {}'.format(np.percentile(review_len_by_token, 75)))

In [ ]:
plt.figure(figsize=(6,4))
plt.boxplot([review_len_by_token],
           labels=['token'],
            showmeans=True)

In [ ]:
plt.figure(figsize=(6,4))
plt.boxplot([review_len_by_eumjeol],
           labels=['Eumjeol'],
           showmeans=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sentiment = train_df['sentiment'].value_counts()
fig, axe = plt.subplots(ncols=1)
fig.set_size_inches(6,3)
sns.countplot(train_df['sentiment'])

157~159 생략

#### 데이터 전처리

In [10]:
import re
import pandas
import numpy
import json
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [ ]:
train_data = pd.read_csv('C:/inkyun/실습파일과 교재/5.자연어처리(실습파일)/dataset/4-1.labeledTrainData.tsv', header = 0, delimiter = '\t', quoting=3)

In [ ]:
review = train_data['review'][0]
review_text = BeautifulSoup(review,'html5lib').get_text()

In [ ]:
review_text = re.sub("[^a-zA-Z]"," ", review_text)

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
review_text = review_text.lower()
words = review_text.split()

In [ ]:
words = [w for w in words if not w in stop_words]

In [ ]:
words

In [ ]:
clean_review = ' '.join(words)
print(clean_review)

In [ ]:
def preprocessing(review, remove_stopwords = False):
    review_text = BeautifulSoup(review,'html5lib').get_text()
    
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    words = review_text.lower().split()
    
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
        clean_review = ' '.join(words)
        
    else:
        clean_review = ' '.join(words)
    
    return clean_review

In [ ]:
clean_train_reviews = []
for review in train_data['review']:
    clean_train_reviews.append(preprocessing(review, remove_stopwords = True))

clean_train_reviews[0]

In [ ]:
clean_train_df = pd.DataFrame({'review':clean_train_reviews, 'sentiment':train_data['sentiment']})

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reviews)

In [ ]:
text_sequences = tokenizer.texts_to_sequences(clean_train_reviews)
text_sequences[0]

In [ ]:
word_vocab = tokenizer.word_index
print(word_vocab)

In [ ]:
idx2word = {v:k for k, v in word_vocab.items()}

In [ ]:
data_configs = {}
data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab) + 1

In [ ]:
MAX_SEQUENCE_LENGTH = 174
train_inputs = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
print('Shape of train data: ', train_inputs.shape)

# 174는 리뷰 길이의 중간값.
# 평균은 이상치에 민감하기 때문에 중간값으로 길이를 맞춘다.

In [ ]:
train_labels = np.array(train_data['sentiment'])
print('Shape of label tensor:', train_labels.shape)

In [ ]:
train_labels

In [ ]:
DATA_IN_PATH = 'c:/inkyun/data/'
TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
TRAIN_CLEAN_DATA = 'train_clean.csv'
DATA_CONFIGS = 'data_configs.json'

import os
if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DATA_IN_PATH)

In [ ]:
# 넘파이 형태 저장
np.save(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'wb'), train_inputs)
np.save(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'wb'), train_labels)

# 정제된 텍스트를 CSV 형태로 저장
clean_train_df.to_csv(DATA_IN_PATH+'train_clean.csv', index = False)

# 딕셔너리를 JSON형태로 저장
json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'), ensure_ascii=False)

## TF-IDF를 활용한 모델 구현(8.4 스킵)

In [ ]:
train_data

In [ ]:
DATA_IN_PATH = 'c:/inkyun/data/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA, header = 0, delimiter = ',', quoting = 3)

In [ ]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

## word2vec을 활용한 모델 구현

In [12]:
DATA_IN_PATH = 'c:/inkyun/data/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA, header = 0, delimiter = ',', quoting = 3)

reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

sentences = []
for review in reviews:
    sentences.append(review.split())

FileNotFoundError: [Errno 2] File c:/inkyun/data/train_clean.csv does not exist: 'c:/inkyun/data/train_clean.csv'

In [ ]:
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

- 아키텍쳐 옵션: skip-gram(default, 느리지만 더 나은 결과) 또는 CBOW.  
- 학습 알고리즘: Hierarchical softmax(default) 또는 negative sampling  
- 다운샘플링: .00001~.001사이의 값 권장  
- num_features: 많은 feature를 사용하는 것이 항상 좋은 것은 아니지만 대체로 조금 더 나은 모델이 된다.  
- context: 알고리즘이 고려해야 하는 context의 단어 수. hierarchical softmax의 경우 대체로 10정도가 적당.
- workers: 실행할 cpu 수
- min_count: 최소 발생 수. 모든 문서에서 여러번 발생하지 않은 단어는 무시된다. 10~100사이가 적당하다.

In [11]:
from gensim.models import word2vec
print('Training model...')
model = word2vec.Word2Vec(sentences,
                          workers = num_workers,
                          size=num_features,
                          min_count = min_word_count,
                          window=context,
                          sample = downsampling)

Training model...


NameError: name 'sentences' is not defined

In [ ]:
model_name = '300features_40minwords_10context'
model.save(model_name)

In [ ]:
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features), dtype=np.float32)
    
    num_words = 0
    
    index2word_set = set(model.wv.index2word)
    
    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])
            
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector